In [2]:
import sqlite3
import os

In [6]:
def create_connection(db_file, verbose = False):
    ''' create a database connection to a SQLITE database'''
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if verbose:
            print(f'Using SQlite Version: {sqlite3.version}')
            print(f'Creating Connection to {db_file}...')
        return conn
    except sqlite3.Error as e:
        print(e)
        

def select_all_query(db_file, query, verbose = False):
    """
    Query all rows in the database Table
    :param  conn: the Connection object
    :return: result as list
    """
    conn = create_connection(db_file, verbose)
    cur = conn.cursor()
    if not query.startswith('SELECT'):
        raise ValueError('Query should begin with SELECT')
        
    cur.execute(query)
    rows = cur.fetchall()
    
    if verbose:
        for row in rows:
            print(row)
    return rows        

In [17]:
"""Listing all the Columns so we know what we're working with"""
con = sqlite3.connect('rpg_db.sqlite3')
c = con.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

[('django_migrations',), ('sqlite_sequence',), ('auth_group',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('armory_item',), ('armory_weapon',), ('django_content_type',), ('auth_permission',), ('auth_user',), ('charactercreator_character_inventory',), ('charactercreator_necromancer',), ('django_session',), ('charactercreator_character',), ('charactercreator_cleric',), ('charactercreator_fighter',), ('charactercreator_mage',), ('charactercreator_thief',)]


In [7]:
"""Question 1: Count of all characters """
results = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_character')
results[0]

(302,)

In [10]:
"""Question 2: How many of each specific subclass? """
results0 = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM charactercreator_cleric, charactercreator_fighter, charactercreator_mage, charactercreator_necromancer, charactercreator_thief')
results0[0]

(308998800,)

In [14]:
"""Question 3: How many total Items?"""
results1 = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM armory_item, armory_weapon, charactercreator_character_inventory')
results1[0]


(5781324,)

In [42]:
"""Question 4: How many of the Items are weapons? How many are not?"""
results2 = select_all_query('rpg_db.sqlite3', 'SELECT COUNT(*) FROM armory_weapon')

results3 = select_all_query('rpg_db.sqlite3', "SELECT COUNT(*) FROM charactercreator_character_inventory")



results2[0], results3

((37,), [(898,)])

In [53]:
"""Question 5 Part 1: How many Items does each character have? (Return first 20 rows)"""

query1 = '''
    SELECT character.name, count(item.item_id) as item_count
        FROM charactercreator_character AS character,
        JOIN charactercreator_character_inventory AS inventory
        ON character.character_id = inventory.character_id
        JOIN armory_item AS item
        ON inventory.item_id = item.item.id
        GROUP BY character.name
        LIMIT 5;
    '''
print('Character | Item')
rows = c.execute(query1)
for row in rows:
    name = row[0]
    print(name, ' | ' ,row[1])

Character | Item


OperationalError: near "JOIN": syntax error

In [27]:
"""Part 2: REturn first 20 rows """
c.execute("SELECT * FROM charactercreator_character_inventory")
c.fetchmany(20)

[(1, 1, 58),
 (2, 1, 20),
 (3, 1, 85),
 (4, 2, 115),
 (5, 2, 93),
 (6, 2, 133),
 (7, 3, 8),
 (8, 3, 43),
 (9, 4, 82),
 (10, 4, 21),
 (11, 4, 85),
 (12, 4, 135),
 (13, 5, 96),
 (14, 5, 145),
 (15, 5, 140),
 (16, 5, 7),
 (17, 6, 93),
 (18, 7, 38),
 (19, 7, 42),
 (20, 7, 123)]

In [39]:
"""Question 6: How many Weapons does each character have? (Return first 20 rows)"""
c.execute("SELECT COUNT(item_id) FROM charactercreator_character_inventory")
c.fetchone()

(898,)

In [50]:
"""Question 7: On average, how many Items does each Character have?"""
c.execute("SELECT AVG(character_id)  FROM charactercreator_character_inventory GROUP BY character_id")
c.fetchone()

(3,)

In [ ]:
"""Question 8: On average, how many Weapons does each character have?"""

In [44]:
"""Part 2, MAking and Populating a Database"""
import pandas as pd
df = pd.read_csv('buddymove_holidayiq.csv')

In [54]:

import sqlite3
import numpy as np

conn = sqlite3.connect('rpg_db.sqlite3')
curs = conn.cursor()

# How many total Characters are there?

query = """SELECT COUNT(character_id)
FROM charactercreator_character AS cc"""

results = curs.execute(query)
print("Total # of characters:", results.fetchall()[0][0])
# How many of each specific subclass?

query = """SELECT COUNT(character_ptr_id)
FROM charactercreator_cleric as cc_cleric"""
results = curs.execute(query)
print("# of clerics:", results.fetchall()[0][0])

query = """SELECT COUNT(character_ptr_id)
FROM charactercreator_fighter"""
results = curs.execute(query)
print("# of fighters:", results.fetchall()[0][0])

query = """SELECT COUNT(character_ptr_id)
FROM charactercreator_mage"""
results = curs.execute(query)
print("# of mages:", results.fetchall()[0][0])

query = """SELECT COUNT(mage_ptr_id)
FROM charactercreator_necromancer"""
results = curs.execute(query)
print("# of necromancers:", results.fetchall()[0][0])

query = """SELECT COUNT(character_ptr_id)
FROM charactercreator_thief"""
results = curs.execute(query)
print("# of thieves:", results.fetchall()[0][0])

# How many total Items?
query = """SELECT COUNT(item_id)
FROM armory_item"""
results = curs.execute(query)
print("# items:", results.fetchall()[0][0])


# How many of the Items are weapons? How many are not?
query = """SELECT COUNT(ai.item_id)
FROM armory_item as ai, armory_weapon as aw
WHERE ai.item_id = aw.item_ptr_id"""
results = curs.execute(query)
print("Weapons: ", results.fetchall()[0][0])

# Not working
query = """SELECT COUNT(item_id)
FROM armory_item
WHERE item_id NOT IN (SELECT item_ptr_id FROM armory_weapon);"""
results = curs.execute(query)
print("Not weapons: ", results.fetchall()[0][0])

# How many Items does each character have? (Return first 20 rows)
query = """SELECT character_id, item_id, COUNT(*)
FROM charactercreator_character_inventory
GROUP BY character_id
LIMIT 20;"""
results = curs.execute(query)
matrix = results.fetchall()
item_list = []
for row in matrix:
    item_list.append(row[2])

print("# of items for first 20 characters: ", item_list)

# How many Weapons does each character have? (Return first 20 rows)
query = """SELECT character_id, item_id, COUNT(*)
FROM charactercreator_character_inventory
WHERE item_id IN (SELECT item_ptr_id FROM armory_weapon)
GROUP BY character_id
LIMIT 20;
"""
results = curs.execute(query)
matrix = results.fetchall()
weapon_list = []
for row in matrix:
    weapon_list.append(row[2])

print("# of weapons for first 20 characters: ", weapon_list)
print(sum(weapon_list)/len(weapon_list))
# On average, how many Items does each Character have?

query = """SELECT AVG(counted) FROM (
SELECT character_id, item_id, COUNT(*) AS counted
FROM charactercreator_character_inventory
GROUP BY character_id);
"""
results = curs.execute(query)
print("Avg. Items per Character: ", results.fetchall()[0][0])

# On average, how many Weapons does each character have?

query = """SELECT AVG(counted) FROM (
SELECT character_id, item_id, COUNT(*) AS counted
FROM charactercreator_character_inventory
WHERE item_id IN (SELECT item_ptr_id FROM armory_weapon)
GROUP BY character_id);
"""
results = curs.execute(query)
print("Avg. Weapons per Character: ", results.fetchall()[0][0])

Total # of characters: 302
# of clerics: 75
# of fighters: 68
# of mages: 108
# of necromancers: 11
# of thieves: 51
# items: 174
Weapons:  37
Not weapons:  137
# of items for first 20 characters:  [3, 3, 2, 4, 4, 1, 5, 3, 4, 4, 3, 3, 4, 4, 4, 1, 5, 5, 3, 1]
# of weapons for first 20 characters:  [2, 1, 1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 2, 3, 2, 2, 2, 1, 1, 1]
1.5
Avg. Items per Character:  2.9735099337748343
Avg. Weapons per Character:  1.3096774193548386
